## v1 pilot

In [18]:
import json

annotator = 'austin'
session = 'XUBUP'

session_data = json.load(open(f'session_data/initial_pilots_5scenarios/annotator_{annotator}-session_{session}.json'))
trajectories = session_data['trajectories']
scenarios = list(trajectories.keys())
print("Loaded trajectories for Scenarios: {}".format(', '.join(scenarios)))

Loaded trajectories for Scenarios: garden000, electronics000, grocery000


In [30]:
scenario ='grocery000'
trajectory = trajectories[scenario]['trajectory']

from pprint import pprint
def print_trajectory(scenario, trajectory):
    pprint(trajectories[scenario]['scenario'])

    for i, t in enumerate(trajectory):
        if t['content'].startswith('ENVIRONMENT OBSERVATION:'):# or (t['role'] == 'assistant' and 'respond' not in t['content'] and 'Buy Now' not in t['content']):
            continue
        print(f"{i}: {t['content']}")
        print("-"*100)
    print("\nTask completed: {}".format(trajectories[scenario]['task_completed']))
    print("Reward: {}".format(trajectories[scenario]['trajectory_reward']))


{'asin': 'B07NDJHPMJ',
 'attributes': ['high fructose', 'low calorie', 'gluten free'],
 'category': 'grocery',
 'goal_options': ['1 pound (pack of 2)', 'medium'],
 'goal_options_dict': {'size': '1 pound (pack of 2)', 'style': 'ghost pepper'},
 'instruction_text': 'You’re planning a game night with friends and need a '
                     'delicious, crowd-pleasing cheese dip that everyone can '
                     'enjoy, including guests with dietary restrictions and '
                     'those mindful of portion sizes. Looking for something '
                     'conveniently packaged and versatile, you want a savory '
                     'option from the deli section that pairs perfectly with '
                     'chips and fresh veggies.',
 'name': 'Mrs. Renfros Ghost Pepper Nacho Cheese Dip Gluten-free (16-oz. jars, '
         '2-pack)',
 'price_upper': 20.0,
 'product_category': 'Grocery & Gourmet Food › Snacks & Sweets › Snack Foods › '
                     'Salsas, Dips

## v2 15 scenarios -- 3 users, 14 sessions

In [2]:
import json
import os

experiments_dir = "session_data/v2_15scenarios"

all_trajectories = []
for file in os.listdir(experiments_dir):
    if file.endswith('.json'):
        with open(os.path.join(experiments_dir, file), 'r') as f:
            data = json.load(f)
            if data['annotator_name'] == 'tejas':
                continue
            for t, v in data['trajectories'].items():
                v['trajectory_id'] = t
                v['annotator_name'] = data['annotator_name']
                if v['task_completed'] != True:
                    continue
                all_trajectories.append(v)

print(f" Loaded {len(all_trajectories)} trajectories")
annotator_names = set([t['annotator_name'] for t in all_trajectories])
print(f" Annotator names: {annotator_names}")
for annotator_name in annotator_names:
    print(f" {annotator_name}: {len([t for t in all_trajectories if t['annotator_name'] == annotator_name])}")

 Loaded 14 trajectories
 Annotator names: {'Zuhair Shaikh', 'WV', 'Darcie'}
 Zuhair Shaikh: 5
 WV: 5
 Darcie: 4


In [82]:
from pprint import pprint
import random

def print_trajectory(trajectory):
    print(f"Annotator: {trajectory['annotator_name']}, trajectory_id: {trajectory['trajectory_id']}")
    print(f"Scenario ID: {trajectory['scenario']['scenario_id']}")
    print(f"Scenario: {trajectory['scenario']['instruction_text']}")
    print(f"\tDesired product ASIN: {trajectory['scenario']['asin']}")
    print(f"\tProduct type: {trajectory['scenario']['product_type']}")
    print(f"\tAttributes: {', '.join(trajectory['scenario']['attributes'])}")
    print(f"\tOptions: {', '.join(trajectory['scenario']['goal_options'])}")
    print(f"\tBudget: ${trajectory['scenario']['price_upper']}")
    print(f"\nTrajectory:")
    for t in trajectory['trajectory']:
        if 'ENVIRONMENT OBSERVATION' in t['content']:
            continue
        if 'ACTION:' in t['content']:
            print("> AGENT: " + t['content'].split('ACTION:')[1].strip())
        else:
            print("> USER: " + t['content'].split('USER SAYS:')[-1].strip())

    print(f"\nNumber of dialog turns: {trajectory['num_dialog_turns']}")
    print(f"Number of agent actions in env: {trajectory['num_environment_steps']}")
    print(f"LLM cost: ${trajectory['llm_cost']:.2f}")
    print(f"\nFinal reward: {trajectory['trajectory_reward']:.4f}")
    print(f"Purchased product:")
    print(f"\tASIN: {trajectory['purchased_item']['asin']}")
    print(f"\tName: {trajectory['purchased_item']['product_name']}")
    print(f"\tAttributes: {', '.join(trajectory['purchased_item']['attributes'])}")
    print("\tOptions: \n" + '\n'.join([f'\t- {k}: {v}' for k, v in trajectory['purchased_item']['options'].items()]) if trajectory['purchased_item']['options'] else "\tOptions: None selected")
    print(f"\tPrice: {trajectory['purchased_item']['price']}")

    print(f"\nUser feedback:")
    print(f"1. Are you satisfied with the product? {trajectory['user_feedback']['product_feedback']}")
    print(f"2. Does the product align with the scenario? {trajectory['user_feedback']['scenario_conformance']}")
    print(f"3. Did you find the assistant helpful? {trajectory['user_feedback']['assistant_helpfulness']}")


trajectory = random.choice(all_trajectories)
print_trajectory(trajectory)


Annotator: Darcie, trajectory_id: U4LYaXzt9n
Scenario ID: fashion002
Scenario: You’re planning to attend a trendy rooftop party in the city and want a chic yet comfortable outfit that will make a stylish statement and pair perfectly with your favorite heels, so you decide to buy a women’s romper.
	Desired product ASIN: B08R76M6D7
	Product type: women's romper
	Attributes: high waist, high heel, loose fit
	Options: black, medium
	Budget: $31.98

Trajectory:
> USER: I am looking to purchase a womens romper for a party
> AGENT: search[womens party romper]
> AGENT: click[B09QLTHKX1]
> AGENT: click[Description]
> AGENT: click[Features]
> AGENT: click[< Prev]
> AGENT: click[Back to Search]
> AGENT: search[womens dressy party romper]
> AGENT: search[womens elegant cocktail romper]
> AGENT: click[B0861KPXKZ]
> AGENT: click[Description]
> AGENT: click[Reviews]
> AGENT: click[< Prev]
> AGENT: click[Back to Search]
> AGENT: search[womens party romper shorts]
> AGENT: click[B08ZN2K853]
> AGENT: cl